In [41]:
import fraud_env
import llmplanner
import pydantic_validator as pv

In [42]:
env = fraud_env.FraudEnv()

# Add banks
env.add_node_with_attribute("bankofamerica", "bank")
env.add_node_with_attribute("chase", "bank")
env.add_node_with_attribute("firstfinancial", "bank")

# Add individuals
env.add_node_with_attribute("olivia", "individual", {"role": "victim"})
env.add_node_with_attribute("betty", "individual", {"role": "victim"})

# Add fraudsters
env.add_node_with_attribute("scamgov", "fraudster", {"role": "fraudco", "status": "active", "description": "Impersonates gov for SID"})
env.add_node_with_attribute("scamco", "fraudster", {"role": "fraudco", "status": "active", "description": "Impersonates gov for SID"})

# Add accounts (using valid banks)
env.add_node_with_attribute("acc_olivia", "account", {"owner": "Olivia", "bank": "BankOfAmerica", "balance": 60000.00})
env.add_node_with_attribute("acc_betty", "account", {"owner": "Betty", "bank": "Chase", "balance": 4000.00})
env.add_node_with_attribute("acc_scamgov", "account", {"owner": "ScamGov", "bank": "FirstFinancial", "balance": 0.00})

# Add ownership edges
env.add_ownership_edge("olivia", "acc_olivia")
env.add_ownership_edge("betty", "acc_betty")
env.add_ownership_edge("scamgov", "acc_scamgov")

Successfully added node bankofamerica as a bank node.
Successfully added node chase as a bank node.
Successfully added node firstfinancial as a bank node.
Successfully added node olivia as a individual node.
Successfully added node betty as a individual node.
Successfully added node scamgov as a fraudster node.
Successfully added node scamco as a fraudster node.
Successfully added node acc_olivia as a account node.
Successfully added node acc_betty as a account node.
Successfully added node acc_scamgov as a account node.
Added ownership relationship between olivia -> acc_olivia
Added ownership relationship between betty -> acc_betty
Added ownership relationship between scamgov -> acc_scamgov


In [43]:
planner = llmplanner.LLMPlanner()
seq = planner.generate_sequence()
print(seq)

{'sequence': ["action(scamgov, sim swap, olivia, call, switched olivia's phone number)", "action(olivia, phone number exposure, scamgov, sms, sent olivia's new phone number)", "action(scamgov, social engineering, bankofamerica, call, tricked bankofamerica into sending olivia's login credentials)", "action(bankofamerica, sensitive info disclosure, scamgov, email, sent olivia's login credentials to scamgov)", 'transaction(acc_olivia, fast payment, acc_scamgov, 3000.00)']}


In [44]:
print(type(seq))

try:
    T = pv.SequenceModel.model_validate(seq, context={"entities": {node[0] for node in env.get_nodes()}})
    print("✅ Validation passed")
except Exception as e:
    print("❌ Validation failed:", e)

<class 'dict'>
✅ Validation passed


In [46]:
env.update_balance("acc_olivia", "acc_scamgov", 1000.00)
print(env.get_nodes())

AttributeError: 'FraudEnv' object has no attribute 'update_balance'